In [1]:
import pandas as pd

## Withdrawal datasets

### BA

In [31]:
df_BA = pd.read_csv('../data/raw_data/withdrawal_BA.csv', sep=';', encoding='utf-8')

In [32]:
def transform_percentage(x):
    y = x.replace(',','.')
    return float(y.strip('%'))/100

In [33]:
df_BA['Isolated_city'] = df_BA['Isolated_city'].apply(transform_percentage)
df_a = df_BA.pivot(index='City Name', columns='Dt')['Isolated_city']
df_a = df_a.T
df_a.index = pd.to_datetime(df_a.index)
df_a = df_a.rename_axis(None)
df_a.columns = map(str.upper, df_a.columns)
df_a

,ABARÉ,ABAÍRA,ACAJUTIBA,ADUSTINA,AIQUARA,ALAGOINHAS,ALCOBAÇA,ALMADINA,AMARGOSA,AMÉLIA RODRIGUES,...,VITÓRIA DA CONQUISTA,VÁRZEA NOVA,VÁRZEA DA ROÇA,VÁRZEA DO POÇO,WAGNER,WANDERLEY,WENCESLAU GUIMARÃES,XIQUE-XIQUE,ÁGUA FRIA,ÉRICO CARDOSO
2021-01-01,0.417,NaN,0.556,NaN,NaN,0.463,0.500,NaN,0.443,0.475,...,0.467,0.480,0.206,NaN,NaN,NaN,0.469,0.368,0.419,NaN
2020-01-02,0.319,0.353,0.331,0.322,0.375,0.291,0.272,0.418,0.273,0.315,...,0.303,0.255,0.386,0.265,0.265,0.290,0.255,0.288,0.318,0.384
2021-01-02,0.444,NaN,0.292,0.370,NaN,0.333,0.273,NaN,0.380,0.385,...,0.363,0.515,0.333,NaN,NaN,NaN,0.455,0.383,NaN,NaN
2020-01-03,0.389,0.333,0.440,0.417,0.423,0.399,0.348,0.395,0.376,0.408,...,0.393,0.392,0.364,0.391,0.407,0.348,0.367,0.351,0.441,0.307
2020-01-04,0.389,0.597,0.439,0.348,0.403,0.425,0.366,0.448,0.413,0.448,...,0.424,0.489,0.328,0.392,0.371,0.386,0.382,0.392,0.357,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-31,0.431,0.472,0.441,0.420,0.536,0.533,0.475,0.600,0.480,0.478,...,0.464,0.459,0.409,0.397,0.476,0.513,0.485,0.472,0.457,0.320
2020-07-31,0.392,0.459,0.385,0.402,0.377,0.380,0.350,0.373,0.410,0.391,...,0.364,0.395,0.368,0.375,0.422,0.358,0.440,0.332,0.425,0.420
2020-08-31,0.407,0.342,0.388,0.384,0.298,0.369,0.406,0.459,0.348,0.344,...,0.353,0.412,0.418,0.347,0.359,0.340,0.358,0.363,0.374,0.421
2020-10-31,0.385,0.345,0.452,0.271,NaN,0.360,0.321,0.435,0.373,0.347,...,0.351,0.338,0.442,0.423,0.432,0.407,0.348,0.441,0.484,0.400


### SP

In [7]:
df_SP = pd.read_csv('../data/raw_data/withdrawal_SP.csv', sep='\t', encoding='utf-16', header=1)
df_b = df_SP.drop(['UF1','Código Município IBGE','População estimada (2020)'], axis=1).T
df_b.columns = df_b.iloc[0]
df_b = df_b.drop(df_b.index[0])
df_b.index = pd.to_datetime(df_b.index, format='%d/%m/%y')

for i in df_b.columns:
    df_b[i] = (df_b[i].str.replace('%',''))
    df_b[i] = df_b[i].astype(float)/100

df_b.columns = map(str.upper, df_b.columns)
df_b

,AMERICANA,AMPARO,ANDRADINA,ARAÇATUBA,ARARAQUARA,ARARAS,ARTUR NOGUEIRA,ARUJÁ,ASSIS,ATIBAIA,...,TATUÍ,TAUBATÉ,TUPÃ,UBATUBA,VALINHOS,VARGEM GRANDE PAULISTA,VÁRZEA PAULISTA,VINHEDO,VOTORANTIM,VOTUPORANGA
2020-02-26,0.36,0.33,0.37,0.32,NaN,0.33,0.37,0.33,0.32,0.33,...,0.32,0.37,0.34,0.35,0.36,0.38,0.31,0.37,0.32,0.35
2020-02-27,0.34,0.33,0.36,NaN,NaN,0.32,0.40,0.32,0.30,0.31,...,0.31,0.36,0.32,0.42,0.34,0.35,NaN,0.34,0.30,0.37
2020-02-28,0.34,0.33,0.36,NaN,NaN,NaN,0.39,0.31,NaN,0.31,...,0.30,0.35,0.31,0.42,0.33,0.34,NaN,0.34,NaN,0.37
2020-02-29,0.35,0.35,0.36,NaN,NaN,0.33,0.42,0.33,0.31,0.34,...,0.32,0.36,0.34,0.41,0.36,0.38,0.33,0.38,0.33,0.37
2020-03-01,0.37,0.36,0.37,0.33,0.33,0.36,0.42,0.37,0.35,0.37,...,0.35,0.38,0.34,0.41,0.39,0.40,0.36,0.42,0.36,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-01,0.42,0.40,0.41,0.39,0.40,0.40,0.50,0.40,0.42,0.43,...,0.43,0.40,0.43,0.51,0.42,0.47,0.42,0.43,0.40,0.42
2021-05-02,0.44,0.45,0.47,0.41,0.45,0.45,0.54,0.42,0.47,0.47,...,0.47,0.44,0.47,0.51,0.46,0.53,0.47,0.47,0.46,0.49
2021-05-03,0.37,0.37,0.39,0.34,0.36,0.37,0.45,0.33,0.40,0.38,...,0.38,0.37,0.40,0.47,0.37,0.45,0.39,0.39,0.37,0.39
2021-05-04,0.37,0.38,0.41,0.34,0.36,0.37,0.46,0.33,0.39,0.39,...,0.40,0.38,0.39,0.49,0.38,0.45,0.39,0.40,0.38,0.41


### States

In [8]:
oi = pd.read_csv('../data/raw_data/withdrawal_states.csv', sep=',', encoding='utf-8')
oi = oi.drop(['Dt-alias', 'AGG(avg_ponderada_br)-alias'], axis=1)
oi = oi.rename(columns={'Dt-value': 'data', 'AGG(avg_ponderada_br)-value': 'withdrawal', 'AGG(state_title)-alias': 'state'})
oi = oi.pivot(index='state', columns='data')['withdrawal'].T
oi.index = pd.to_datetime(oi.index)
oi = oi.rename_axis(None)
oi.columns = map(str.upper, oi.columns)

oi

,ACRE,ALAGOAS,AMAPÁ,AMAZONAS,BAHIA,CEARÁ,DISTRITO FEDERAL,ESPÍRITO SANTO,GOIÁS,MARANHÃO,...,PIAUÍ,RIO GRANDE DO NORTE,RIO GRANDE DO SUL,RIO DE JANEIRO,RONDÔNIA,RORAIMA,SANTA CATARINA,SERGIPE,SÃO PAULO,TOCANTINS
2020-02-01,0.362524,0.314066,0.328219,0.348006,0.306312,0.325898,0.310840,0.310485,0.302212,0.321198,...,0.304575,0.309341,0.328249,0.323001,0.338244,0.328113,0.322547,0.302363,0.317490,0.293416
2020-02-02,0.422642,0.379520,0.393595,0.409687,0.369962,0.386480,0.365940,0.377299,0.359868,0.372274,...,0.360967,0.368838,0.392126,0.390020,0.407545,0.382002,0.396528,0.357507,0.386574,0.362883
2020-02-03,0.296582,0.241381,0.260747,0.294926,0.249951,0.262221,0.259102,0.258660,0.249188,0.257573,...,0.249871,0.250255,0.267627,0.274816,0.272736,0.249112,0.272453,0.243980,0.264410,0.236162
2020-02-04,0.305246,0.243551,0.333012,0.306876,0.255916,0.283145,0.266704,0.264473,0.260905,0.278621,...,0.263682,0.256670,0.268206,0.285493,0.288542,0.267635,0.294200,0.261422,0.261711,0.248861
2020-02-05,0.311875,0.252338,0.277639,0.302987,0.258713,0.285186,0.265072,0.264230,0.262661,0.288194,...,0.269003,0.261507,0.280892,0.291794,0.293837,0.272665,0.284608,0.259066,0.264772,0.246131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-18,0.386937,0.347557,0.420413,0.405499,0.364423,0.417557,0.362507,0.346170,0.334409,0.369809,...,0.448757,0.358560,0.337077,0.326447,0.371017,0.358590,0.306929,0.352469,0.362939,0.316856
2021-03-19,0.399777,0.348309,0.460930,0.385138,0.372883,0.492154,0.366548,0.344548,0.354499,0.390104,...,0.413853,0.356629,0.332403,0.332790,0.374544,0.367148,0.311389,0.356639,0.358957,0.379078
2021-03-20,0.550633,0.466370,0.581655,0.409639,0.450641,0.456867,0.422031,0.438710,0.407752,0.429857,...,0.462247,0.467697,0.473883,0.396314,0.467073,0.373418,0.385628,0.420965,0.448898,0.362319
2021-03-21,0.641509,0.530730,0.564767,0.462841,0.531088,0.510622,0.477823,0.531664,0.460153,0.472727,...,0.477432,0.557071,0.569197,0.459895,0.566298,0.435374,0.492128,0.494390,0.514227,0.405970


## Simple data cities

In [9]:
df_GDP = pd.read_csv('../data/raw_data/GDP.csv', sep=';', header=4)
df_GDP = df_GDP.iloc[:-12]
df_GDP = df_GDP.rename(columns={'1': 'city_code', 'Brasil': 'city_name', '7004141000': 'gdp'})
a = df_GDP['city_name'].str.split(' \(', expand=True)
df_GDP['state'] = a[1].str.replace('\)', '')
df_GDP['city_name'] = a[0].apply(str.upper)
df_GDP_gross = df_GDP
df_GDP = df_GDP[(df_GDP['state']=='BA') | (df_GDP['state']=='SP')]
df_GDP['city_code'] = df_GDP['city_code'].astype(int)
# df_GDP = df_GDP.set_index('city_code')
df_GDP

/Users/lucasagrizzi/opt/anaconda3/envs/study/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  """
/Users/lucasagrizzi/opt/anaconda3/envs/study/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,city_code,city_name,gdp,state
1827,2900108,ABAÍRA,59565.0,BA
1828,2900207,ABARÉ,137847.0,BA
1829,2900306,ACAJUTIBA,133899.0,BA
1830,2900355,ADUSTINA,112228.0,BA
1831,2900405,ÁGUA FRIA,110270.0,BA
...,...,...,...,...
3907,3557006,VOTORANTIM,3093368.0,SP
3908,3557105,VOTUPORANGA,2928688.0,SP
3909,3557154,ZACARIAS,88517.0,SP
3910,3557204,CHAVANTES,207456.0,SP


In [10]:
df_pop = pd.read_csv('../data/raw_data/population.csv')
df_pop = df_pop.drop(columns=['Unnamed: 0', 'year'])
df_pop['city_code'].astype(int)
# df_pop = df_pop.set_index('city_code')
df_pop

,city_code,population,area
0,3166600,786,335.659
1,3507209,836,118.951
2,5101209,956,688.326
3,4306924,1088,71.191
4,1715507,1106,209.292
...,...,...,...
5565,2304400,2643247,312.353
5566,2927408,2857329,693.453
5567,5300108,2974703,5760.783
5568,3304557,6688927,1200.329


In [11]:
df_cities = pd.merge(df_GDP, df_pop, on='city_code', how='inner')
df_cities

,city_code,city_name,gdp,state,population,area
0,2900108,ABAÍRA,59565.0,BA,8767,538.677
1,2900207,ABARÉ,137847.0,BA,19814,1604.923
2,2900306,ACAJUTIBA,133899.0,BA,15129,181.475
3,2900355,ADUSTINA,112228.0,BA,16784,629.099
4,2900405,ÁGUA FRIA,110270.0,BA,16901,742.775
...,...,...,...,...,...,...
1057,3557006,VOTORANTIM,3093368.0,SP,121331,183.517
1058,3557105,VOTUPORANGA,2928688.0,SP,93736,420.703
1059,3557154,ZACARIAS,88517.0,SP,2684,319.056
1060,3557204,CHAVANTES,207456.0,SP,12418,188.727


In [12]:
df_san = pd.read_csv('../data/raw_data/sanitation.csv', sep=';', header=4)
df_san = df_san.rename(columns={'Cód.': 'city_code'})
df_san = df_san.iloc[1:-12].drop(columns='Brasil e Município')

df_san = df_san.astype(str)
for i in df_san.columns:
    df_san[i] = df_san[i].str.replace('-', '0')

df_san = df_san.astype(float)
df_san

,city_code,Total geral de municípios,Total com algum serviço de saneamento básico,Rede geral de distribuição de água,Rede coletora de esgoto,Manejo de resíduos sólidos,Manejo de águas pluviais
1,1100015.0,1.0,1.0,1.0,0.0,1.0,1.0
2,1100023.0,1.0,1.0,1.0,0.0,1.0,1.0
3,1100031.0,1.0,1.0,1.0,0.0,1.0,0.0
4,1100049.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1100056.0,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...
5560,5222005.0,1.0,1.0,1.0,0.0,1.0,1.0
5561,5222054.0,1.0,1.0,1.0,0.0,1.0,1.0
5562,5222203.0,1.0,1.0,1.0,0.0,1.0,1.0
5563,5222302.0,1.0,1.0,1.0,0.0,1.0,1.0


In [13]:
df_cities = pd.merge(df_cities, df_san, on='city_code', how='inner')
df_cities

,city_code,city_name,gdp,state,population,area,Total geral de municípios,Total com algum serviço de saneamento básico,Rede geral de distribuição de água,Rede coletora de esgoto,Manejo de resíduos sólidos,Manejo de águas pluviais
0,2900108,ABAÍRA,59565.0,BA,8767,538.677,1.0,1.0,1.0,1.0,1.0,1.0
1,2900207,ABARÉ,137847.0,BA,19814,1604.923,1.0,1.0,1.0,1.0,1.0,1.0
2,2900306,ACAJUTIBA,133899.0,BA,15129,181.475,1.0,1.0,1.0,0.0,1.0,1.0
3,2900355,ADUSTINA,112228.0,BA,16784,629.099,1.0,1.0,1.0,1.0,1.0,1.0
4,2900405,ÁGUA FRIA,110270.0,BA,16901,742.775,1.0,1.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1057,3557006,VOTORANTIM,3093368.0,SP,121331,183.517,1.0,1.0,1.0,1.0,1.0,1.0
1058,3557105,VOTUPORANGA,2928688.0,SP,93736,420.703,1.0,1.0,1.0,1.0,1.0,1.0
1059,3557154,ZACARIAS,88517.0,SP,2684,319.056,1.0,1.0,1.0,1.0,1.0,1.0
1060,3557204,CHAVANTES,207456.0,SP,12418,188.727,1.0,1.0,1.0,1.0,1.0,1.0


In [14]:
df_urb = pd.read_csv('../data/raw_data/urbanization.csv', sep=';', header=4)
df_urb = df_urb.rename(columns={'Cód.': 'city_code', 'Urbana': 'urbanization'})
df_urb = df_urb.iloc[1:-12].drop(columns=['Brasil e Município', 'Número de moradores'])
df_urb['urbanization'] = df_urb['urbanization'].str.replace(',','.')
df_urb['urbanization'] = pd.to_numeric(df_urb['urbanization'], downcast='float')
df_urb['city_code'] = df_urb['city_code'].astype(int)
df_urb

,city_code,urbanization
1,1100015,59.040001
2,1100023,84.360001
3,1100031,44.810001
4,1100049,80.050003
5,1100056,85.019997
...,...,...
5561,5222005,73.660004
5562,5222054,85.110001
5563,5222203,71.919998
5564,5222302,31.440001


In [15]:
df_cities = pd.merge(df_cities, df_urb, on='city_code', how='inner')
df_cities

,city_code,city_name,gdp,state,population,area,Total geral de municípios,Total com algum serviço de saneamento básico,Rede geral de distribuição de água,Rede coletora de esgoto,Manejo de resíduos sólidos,Manejo de águas pluviais,urbanization
0,2900108,ABAÍRA,59565.0,BA,8767,538.677,1.0,1.0,1.0,1.0,1.0,1.0,46.349998
1,2900207,ABARÉ,137847.0,BA,19814,1604.923,1.0,1.0,1.0,1.0,1.0,1.0,54.889999
2,2900306,ACAJUTIBA,133899.0,BA,15129,181.475,1.0,1.0,1.0,0.0,1.0,1.0,88.300003
3,2900355,ADUSTINA,112228.0,BA,16784,629.099,1.0,1.0,1.0,1.0,1.0,1.0,37.389999
4,2900405,ÁGUA FRIA,110270.0,BA,16901,742.775,1.0,1.0,1.0,0.0,1.0,1.0,39.720001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,3557006,VOTORANTIM,3093368.0,SP,121331,183.517,1.0,1.0,1.0,1.0,1.0,1.0,96.389999
1058,3557105,VOTUPORANGA,2928688.0,SP,93736,420.703,1.0,1.0,1.0,1.0,1.0,1.0,97.279999
1059,3557154,ZACARIAS,88517.0,SP,2684,319.056,1.0,1.0,1.0,1.0,1.0,1.0,79.849998
1060,3557204,CHAVANTES,207456.0,SP,12418,188.727,1.0,1.0,1.0,1.0,1.0,1.0,92.879997


In [16]:
df_work = pd.read_csv('../data/raw_data/work_type.csv', sep=';', header=4)
df_work = df_work.rename(columns={'Cód.': 'city_code'})
df_work = df_work.iloc[24:-12].drop(columns=['Brasil e Município', 'Posição na ocupação, subgrupo e categoria do emprego e contribuição para instituto de previdência oficial no trabalho principal'])
df_work['Total'] = df_work['Total'].str.replace(',','.')
df_work['Total'] = df_work['Total'].str.replace('-','0')
df_work['Total'] = pd.to_numeric(df_work['Total'], downcast='float')
df_work['city_code'] = df_work['city_code'].astype(int)
df_work = df_work.pivot(index='city_code', columns='Seção de atividade do trabalho principal')['Total']
df_work

Seção de atividade do trabalho principal,"Administração pública, defesa e seguridade social","Agricultura, pecuária, produção florestal, pesca e aquicultura",Alojamento e alimentação,"Artes, cultura, esporte e recreação",Atividades administrativas e serviços complementares,"Atividades financeiras, de seguros e serviços relacionados",Atividades imobiliárias,Atividades mal especificadas,"Atividades profissionais, científicas e técnicas",Comércio; reparação de veículos automotores e motocicletas,...,Indústrias de transformação,Indústrias extrativas,Informação e comunicação,Organismos internacionais e outras instituições extraterritoriais,Outras atividades de serviços,Saúde humana e serviços sociais,Serviços domésticos,Total,"Transporte, armazenagem e correio","Água, esgoto, atividades de gestão de resíduos e descontaminação"
city_code,,,,,,,,,,,,,,,,,,,,,
1100015,4.11,39.259998,1.46,0.27,1.36,0.33,0.00,5.48,1.05,14.900000,...,3.14,0.00,0.63,0.00,1.62,2.79,8.10,100.0,3.20,0.12
1100023,4.44,15.680000,3.16,0.41,3.36,0.66,0.19,5.31,2.67,20.040001,...,8.95,3.32,0.62,0.00,2.57,2.82,7.13,100.0,4.12,0.50
1100031,7.52,46.150002,2.30,0.18,1.40,0.38,0.00,2.79,1.04,7.410000,...,4.63,0.00,0.29,0.00,1.08,4.54,9.03,100.0,1.40,0.40
1100049,5.13,20.540001,3.58,0.41,2.16,0.79,0.07,3.96,2.40,20.120001,...,10.10,0.23,0.89,0.00,2.24,3.42,6.63,100.0,3.51,0.55
1100056,6.78,27.030001,2.07,0.20,1.93,0.47,0.00,5.05,1.98,15.460000,...,6.77,0.06,0.70,0.00,3.52,1.38,9.13,100.0,3.65,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5222005,4.91,24.410000,3.85,0.59,0.97,0.41,0.06,3.61,1.39,15.140000,...,13.85,0.15,0.17,0.00,2.87,2.03,9.37,100.0,3.83,0.54
5222054,7.23,31.510000,2.84,0.35,0.71,0.64,0.19,1.78,1.00,14.500000,...,15.40,0.25,0.36,0.00,1.99,1.82,7.40,100.0,3.18,0.15
5222203,11.71,26.340000,3.24,0.17,0.35,0.00,0.00,15.79,0.20,9.260000,...,14.77,0.17,0.39,0.00,1.25,0.67,5.65,100.0,1.31,0.20


In [17]:
df_cities = pd.merge(df_cities, df_work, on='city_code', how='inner')
df_cities

,city_code,city_name,gdp,state,population,area,Total geral de municípios,Total com algum serviço de saneamento básico,Rede geral de distribuição de água,Rede coletora de esgoto,...,Indústrias de transformação,Indústrias extrativas,Informação e comunicação,Organismos internacionais e outras instituições extraterritoriais,Outras atividades de serviços,Saúde humana e serviços sociais,Serviços domésticos,Total,"Transporte, armazenagem e correio","Água, esgoto, atividades de gestão de resíduos e descontaminação"
0,2900108,ABAÍRA,59565.0,BA,8767,538.677,1.0,1.0,1.0,1.0,...,1.230000,0.08,0.11,0.0,0.32,4.08,4.49,100.0,2.49,0.32
1,2900207,ABARÉ,137847.0,BA,19814,1604.923,1.0,1.0,1.0,1.0,...,2.160000,0.00,0.00,0.0,1.04,2.18,3.67,100.0,1.31,0.65
2,2900306,ACAJUTIBA,133899.0,BA,15129,181.475,1.0,1.0,1.0,0.0,...,3.690000,0.00,0.20,0.0,1.89,1.83,5.78,100.0,3.09,0.65
3,2900355,ADUSTINA,112228.0,BA,16784,629.099,1.0,1.0,1.0,1.0,...,1.440000,0.09,0.07,0.0,0.34,3.13,3.97,100.0,0.70,0.71
4,2900405,ÁGUA FRIA,110270.0,BA,16901,742.775,1.0,1.0,1.0,0.0,...,1.700000,0.00,0.16,0.0,0.92,1.34,4.09,100.0,2.25,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,3557006,VOTORANTIM,3093368.0,SP,121331,183.517,1.0,1.0,1.0,1.0,...,23.830000,0.31,1.60,0.0,2.96,3.49,5.85,100.0,4.42,1.21
1058,3557105,VOTUPORANGA,2928688.0,SP,93736,420.703,1.0,1.0,1.0,1.0,...,21.570000,0.15,0.89,0.0,3.00,5.36,7.50,100.0,3.10,0.72
1059,3557154,ZACARIAS,88517.0,SP,2684,319.056,1.0,1.0,1.0,1.0,...,17.469999,0.00,0.21,0.0,1.07,2.48,5.60,100.0,1.40,0.00
1060,3557204,CHAVANTES,207456.0,SP,12418,188.727,1.0,1.0,1.0,1.0,...,33.980000,0.08,0.68,0.0,1.86,3.19,8.01,100.0,1.52,0.39


## Variable in Time data Cities

In [37]:
df_tv = pd.read_csv('../data/raw_data/cable_television.csv')
df_tv = df_tv.drop(columns=['Unnamed: 0', 'city', 'state'])
df_tv = df_tv.rename(columns={'accesses': 'cable_tv_accesses'})
df_tv

,city_code,date,cable_tv_accesses
0,5200050,2020-01-01,290
1,5200050,2020-02-01,278
2,5200050,2020-03-01,277
3,5200050,2020-04-01,269
4,5200050,2020-05-01,247
...,...,...,...
83542,3533809,2020-11-01,98
83543,3533809,2020-12-01,97
83544,3533809,2021-01-01,103
83545,3533809,2021-02-01,110


In [38]:
df_band = pd.read_csv('../data/raw_data/fixed_bandwidth.csv')
df_band = df_band.drop(columns=['Unnamed: 0', 'city', 'state'])
df_band = df_band.rename(columns={'accesses': 'fixed_bandwidth_accesses'})

df_band

,city_code,date,fixed_bandwidth_accesses
0,5200050,2020-01-01,546
1,5200050,2020-02-01,529
2,5200050,2020-03-01,1008
3,5200050,2020-04-01,537
4,5200050,2020-05-01,518
...,...,...,...
83545,3533809,2020-11-01,437
83546,3533809,2020-12-01,442
83547,3533809,2021-01-01,489
83548,3533809,2021-02-01,489


In [39]:
df_fixtel = pd.read_csv('../data/raw_data/fixed_telephony.csv')
df_fixtel = df_fixtel.drop(columns=['Unnamed: 0', 'city', 'state'])
df_fixtel['city_code'] = df_fixtel['city_code'].astype(int)
df_fixtel = df_fixtel.rename(columns={'accesses': 'fixed_telephony_accesses'})
df_fixtel

,city_code,date,fixed_telephony_accesses
0,5200050,2020-01-01,1603
1,5200050,2020-02-01,1571
2,5200050,2020-03-01,1536
3,5200050,2020-04-01,1512
4,5200050,2020-05-01,1474
...,...,...,...
83536,3533809,2020-11-01,576
83537,3533809,2020-12-01,555
83538,3533809,2021-01-01,217
83539,3533809,2021-02-01,214


In [34]:
# SE PA JUNTAR TODAS AS TECNOLOGIAS

In [40]:
df_mobtel = pd.read_csv('../data/raw_data/mobile_telephony.csv')
df_mobtel = df_mobtel.drop(columns=['Unnamed: 0', 'city', 'state'])
df_mobtel = df_mobtel.rename(columns={'accesses': 'mobile_telephony_accesses'})
df_mobtel[df_mobtel['city_code']==3550308]

,city_code,date,product_type,technology,mobile_telephony_accesses
779283,3550308,2020-01-01,data,2G,18996
779284,3550308,2020-01-01,data,3G,475041
779285,3550308,2020-01-01,data,4G,6237023
779286,3550308,2020-01-01,other,2G,1552857
779287,3550308,2020-01-01,other,3G,700796
...,...,...,...,...,...
779525,3550308,2021-03-01,voice,3G,57833
779526,3550308,2021-03-01,voice,4G,242726
779527,3550308,2021-03-01,voice+data,2G,690359
779528,3550308,2021-03-01,voice+data,3G,1159515


In [44]:
df_time_cities = pd.merge(df_tv, df_band, on=['city_code', 'date'], how='inner')
df_time_cities = pd.merge(df_time_cities, df_fixtel, on=['city_code', 'date'], how='inner')
# df_time_cities = pd.merge(df_time_cities, df_mobtel, on=['city_code', 'date'], how='inner')

df_time_cities

,city_code,date,cable_tv_accesses,fixed_bandwidth_accesses,fixed_telephony_accesses
0,5200050,2020-01-01,290,546,1603
1,5200050,2020-02-01,278,529,1571
2,5200050,2020-03-01,277,1008,1536
3,5200050,2020-04-01,269,537,1512
4,5200050,2020-05-01,247,518,1474
...,...,...,...,...,...
83533,3533809,2020-11-01,98,437,576
83534,3533809,2020-12-01,97,442,555
83535,3533809,2021-01-01,103,489,217
83536,3533809,2021-02-01,110,489,214
